
##### بِسْمِ اللهِ

In [1]:
import gymnasium as gym
import numpy as np
import torch
from IPython import display
import torch.nn as nn 
import torch.optim as optim
from torch.distributions import Categorical 

%matplotlib inline


In [2]:
# initializing the environment - cartpole
# 1 loading the cartpole environment 
cartpole_env=gym.make('CartPole-v1',render_mode='human')
print(cartpole_env.action_space)
print(cartpole_env.observation_space)


Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [3]:
# 2 Try to sample random actions using env.action_space.sample() and observe how many 
# timesteps are taken in each episode
cartpole_env.reset(seed=58)

(array([-0.01909882,  0.03798336, -0.02569086, -0.02915795], dtype=float32),
 {})

In [4]:
cartpole_env.action_space.seed(58)

[58]

In [ ]:
def reinforce(env,param,alpha,episodes):
    reward_data=[]
    for each_episode in range(episodes):
        observation = env.reset()[0]
        episode_data = []
        
        done = False
        total_reward=0
        while not done:
            cartpole_env.render()
            
            # calculating the policy value 
            policy_value = np.dot(param, observation)
            # apply sigmoid function to get the probability 
            prob = 1 / (1 + np.exp(-policy_value))
            # print("The probability is and the type of probability is: ",prob,type(prob))

            if prob > 0.5:
                action = 1
            else:
                action = 0

            # get the next observation and reward 
            next_observation, reward, done, _, _ = env.step(action)
            episode_data.append((observation, action, reward, prob))
            total_reward+=reward
            observation = next_observation
            

        # updating the policy parameters 
        for i in range(len(episode_data)):
            obs, act, rew, prob = episode_data[i]
            # applying the update rule
            param = param + alpha * (1 - prob) * obs * rew
        
        reward_data.append(total_reward)

    return param,reward_data




In [ ]:
cartpole_env.reset()
cartpole_env_size=cartpole_env.observation_space.shape[0]
alpha=0.001
episodes=10

policy_params=np.array(np.random.rand(4)*2-1)
updated_policy_params,reward_data=reinforce(cartpole_env,policy_params,alpha,episodes)
print(f"The initial Policy Parameters were {policy_params} and the updated Policy Params were {updated_policy_params}")
print("The rewards obtained are : ",reward_data)
print(f"The Max reward obtained is : {max(reward_data)} and the Min reward obtained is : {min(reward_data)}")

episodes_datapoints=np.arange(0,episodes,1)
plt.plot(episodes_datapoints,reward_data)
